# Main

In [14]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV

from catboost import CatBoostClassifier

In [15]:
def label_encode(train_data, test_data, columns):
    'Returns a DataFrame with encoded columns'
    encoded_cols = []
    for col in columns:
        factorised = pd.factorize(train_data[col])[1]
        labels = pd.Series(range(len(factorised)), index=factorised)
        encoded_col_train = train_data[col].map(labels) 
        encoded_col_test = test_data[col].map(labels)
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = -1
        encoded_cols.append(pd.DataFrame({'label_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

def freq_encode(train_data, test_data, columns):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    nsamples = train_data.shape[0]
    for col in columns:    
        freqs_cat = train_data.groupby(col)[col].count()/nsamples
        encoded_col_train = train_data[col].map(freqs_cat)
        encoded_col_test = test_data[col].map(freqs_cat)
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = 0
        encoded_cols.append(pd.DataFrame({'freq_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

def mean_encode(train_data, test_data, columns, target_col, reg_method=None,
                alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()
        target_means_cats = train_data.groupby(col)[target_col].mean()
        target_means_cats_adj = (target_means_cats*nrows_cat + 
                                 target_mean_global*alpha)/(nrows_cat+alpha)
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)
        # Getting a train encodings
        if reg_method == 'expanding_mean':
            train_data_shuffled = train_data.sample(frac=1, random_state=1)
            cumsum = train_data_shuffled.groupby(col)[target_col].cumsum() - train_data_shuffled[target_col]
            cumcnt = train_data_shuffled.groupby(col).cumcount()
            encoded_col_train = cumsum/(cumcnt)
            encoded_col_train.fillna(target_mean_global, inplace=True)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))
        elif (reg_method == 'k_fold') and (folds > 1):
            kfold = StratifiedKFold(folds, shuffle=True, random_state=1)
            parts = []
            for tr_in, val_ind in kfold.split(train_data[columns].values, train_data[target_col].values):
                # divide data
                df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
                # getting means on data for estimation (all folds except estimated)
                nrows_cat = df_for_estimation.groupby(col)[target_col].count()
                target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
                target_means_cats_adj = (target_means_cats*nrows_cat + 
                                         target_mean_global*alpha)/(nrows_cat+alpha)
                # Mapping means to estimated fold
                encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
                if add_random:
                    encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, 
                                                                             size=(encoded_col_train_part.shape[0]))
                # Saving estimated encodings for a fold
                parts.append(encoded_col_train_part)
            encoded_col_train = pd.concat(parts, axis=0)
            encoded_col_train.fillna(target_mean_global, inplace=True)
        else:
            encoded_col_train = train_data[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))

        # Saving the column with means
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = target_mean_global
        encoded_cols.append(pd.DataFrame({'mean_'+target_col+'_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

In [101]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16) 
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64) 
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [173]:
PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')

train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')
test_df.index = range(100000, 200000)

air = pd.read_csv(PATH_TO_DATA / 'airports.csv')
air = air.set_index('IATA_CODE')

train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [174]:
air_city = air['CITY'].to_dict()
air_state = air['STATE'].to_dict()
air_country = air['COUNTRY'].to_dict()
air_lat = air['LATITUDE'].to_dict()
air_long = air['LONGITUDE'].to_dict()

# Preproc

In [175]:
for df in [train_df, test_df]:
    for col in ['Origin', 'Dest']:
        df[col+'CITY'] = df[col].map(air_city).fillna('other')
        df[col+'STATE'] = df[col].map(air_state).fillna('other')
    df['flight'] = df['Origin'] + '-->' + df['Dest']
    df['flight_city'] = df['OriginCITY'] + '-->' + df['DestCITY']
    df['flight_state'] = df['OriginSTATE'] + '-->' + df['DestSTATE']

unique_carrier = list(set(train_df['UniqueCarrier']) & set(test_df['UniqueCarrier']))
or_dest = list(set(train_df['Origin']) & set(test_df['Origin']) & set(train_df['Dest']) & set(test_df['Dest']))
flight_set = list(set(train_df['flight']) & set(test_df['flight']))
flight_city = list(set(train_df['flight_city']) & set(test_df['flight_city']))
flight_state = list(set(train_df['flight_state']) & set(test_df['flight_state']))

In [176]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,OriginCITY,OriginSTATE,DestCITY,DestSTATE,flight,flight_city,flight_state
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,Atlanta,GA,Dallas-Fort Worth,TX,ATL-->DFW,Atlanta-->Dallas-Fort Worth,GA-->TX
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,Pittsburgh,PA,Orlando,FL,PIT-->MCO,Pittsburgh-->Orlando,PA-->FL
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,Raleigh,NC,Cleveland,OH,RDU-->CLE,Raleigh-->Cleveland,NC-->OH
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,Denver,CO,Memphis,TN,DEN-->MEM,Denver-->Memphis,CO-->TN
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,Chicago,IL,Omaha,NE,MDW-->OMA,Chicago-->Omaha,IL-->NE


In [182]:
minutes_in_day = 24 * 60

def upd_time(val):
    hour = val // 100
    minutes = val % 100
    if hour >= 24:
        hour = hour - 24
    return hour * 60 + minutes

def preproc(df, test=False):
    df['Month'] = df['Month'].apply(lambda x: int(x[2:]))
    df['DayofMonth'] = df['DayofMonth'].apply(lambda x: int(x[2:]))
    df['DayOfWeek'] = df['DayOfWeek'].apply(lambda x: int(x[2:]))
    df['Holiday'] = (df['DayOfWeek'] > 5).astype('int')
    
    df['DepTime'] = df['DepTime'].apply(upd_time)
    df['sin_time'] = np.sin(2 * np.pi * df['DepTime'] / minutes_in_day)
    df['cos_time'] = np.cos(2 * np.pi * df['DepTime'] / minutes_in_day)
    df['hour'] = df['DepTime'] // 60
    df['minutes'] = df['DepTime'] % 60
    
    df['morning'] = ((df['hour'] >= 6) & (df['hour'] < 12)).astype('int')
    df['day'] = ((df['hour'] >= 12) & (df['hour'] < 18)).astype('int')
    df['evening'] = ((df['hour'] >= 18) & (df['hour'] < 24)).astype('int')
    df['night'] = ((df['hour'] >= 0) & (df['hour'] < 6)).astype('int')
    df['dtimeLabel'] = df['morning'] + df['day'] * 2 + df['evening'] * 3 + df['night'] * 4
    
    df['low_delay'] = ((df['hour'] >= 4) & (df['hour'] < 9)).astype('int')
    df['other_time'] = ((df['hour'] < 9) & (df['hour'] >=5)).astype('int')
    df['delay_time'] = ((df['hour'] >= 13) & (df['hour'] < 24) | (df['hour'] < 5)).astype('int')
    df['middle_time'] = ((df['hour'] >= 9) & (df['hour'] < 13)).astype('int')
    
    df['hour_bins'] = pd.cut(df['hour'], [-1, 5, 8, 10, 12, 50], right=False)
    df['daytime'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 23], include_lowest=True)

    df['winter'] = ((df['Month'].isin([12, 1, 2]))).astype('int')
    df['spring'] = ((df['Month'].isin([3, 4, 5]))).astype('int')
    df['summer'] = ((df['Month'].isin([6, 7, 8]))).astype('int')
    df['autumn'] = ((df['Month'].isin([9, 10, 11]))).astype('int')
    df['4seasonsLabel'] = df['winter'] + df['spring'] * 2 + df['summer'] * 3 + df['autumn'] * 4

    df['Distance_log'] = df['Distance'].apply(np.log)
    
    for col in ['Origin', 'Dest']:
        df[col+'CITY'] = df[col].map(air_city).fillna('other')
        df[col+'STATE'] = df[col].map(air_state).fillna('other')
        df[col+'COUNTRY'] = df[col].map(air_country).fillna('other')

        df[col+'LATITUDE'] = df[col].map(air_lat)
        df[col+'LONGITUDE'] = df[col].map(air_long)

        df[col+'x_coord'] = np.cos(df[col+'LATITUDE']) * np.cos(df[col+'LONGITUDE'])
        df[col+'y_coord'] = np.cos(df[col+'LATITUDE']) * np.sin(df[col+'LONGITUDE'])
        df[col+'z_coord'] = np.sin(df[col+'LATITUDE'])
        
    df['x_coord'] = df['Originx_coord'] - df['Destx_coord']
    df['y_coord'] = df['Originy_coord'] - df['Desty_coord']
    df['z_coord'] = df['Originz_coord'] - df['Destz_coord']
    df['LATITUDE'] = df['OriginLATITUDE'] - df['DestLATITUDE']
    df['LONGITUDE'] = df['OriginLONGITUDE'] - df['DestLONGITUDE']
    
    df['flight'] = df['Origin'] + '-->' + df['Dest']
    df['flight_city'] = df['OriginCITY'] + '-->' + df['DestCITY']
    df['flight_state'] = df['OriginSTATE'] + '-->' + df['DestSTATE']
    
    df.loc[~df['Origin'].isin(or_dest), 'Origin'] = 'other'
    df.loc[~df['Dest'].isin(or_dest), 'Dest'] = 'other'
    df.loc[~df['UniqueCarrier'].isin(unique_carrier), 'UniqueCarrier'] = 'other'
    df.loc[~df['flight'].isin(flight_set), 'flight'] = 'other'
    df.loc[~df['flight_city'].isin(flight_city), 'flight_city'] = 'other'
    df.loc[~df['flight_state'].isin(flight_state), 'flight_state'] = 'other'
    
    if not test:
        le_carrier.fit(df['UniqueCarrier'])
        le_or_dest.fit(pd.concat([df['Origin'], df['Dest']]))
        le_flight.fit(df['flight'])
        le_flight_city.fit(df['flight_city'])
        le_flight_state.fit(df['flight_state'])
        le_hour.fit(df['hour_bins'])
        le_daytime.fit(df['daytime'])
    
    df['UniqueCarrierLabel'] = le_carrier.transform(df['UniqueCarrier'])
    df['OriginLabel'] = le_or_dest.transform(df['Origin'])
    df['DestLabel'] = le_or_dest.transform(df['Dest'])
    df['flightLabel'] = le_flight.transform(df['flight'])
    df['flight_cityLabel'] = le_flight_city.transform(df['flight_city'])
    df['flight_stateLabel'] = le_flight_state.transform(df['flight_state'])
    df['hour_binsLabel'] = le_hour.transform(df['hour_bins'])
    df['daytimeLabel'] = le_hour.transform(df['daytime'])

    for col in df.columns:
        if df[col].isna().sum() > 0:
            if df[col].dtype != 'object':
                df[col] = df[col].fillna(df[col].median())
            else:
                common_val = df[col].value_counts().index[0]
                df[col] = df[col].fillna(common_val)
    
    return df

le_carrier = LabelEncoder()
le_or_dest = LabelEncoder()
le_target = LabelEncoder()
le_flight = LabelEncoder()
le_flight_city = LabelEncoder()
le_flight_state = LabelEncoder()
le_hour = LabelEncoder()
le_daytime = LabelEncoder()

In [183]:
train_df['dep_delayed_15min'] = le_target.fit_transform(train_df['dep_delayed_15min'])
categ_cols = ['UniqueCarrier', 'Origin', 'Dest', 'flight']

train_df_fe, test_df_fe = freq_encode(train_df, test_df, columns=categ_cols)

train_df_mekf, test_df_mekf = mean_encode(train_df, test_df, columns=categ_cols, target_col='dep_delayed_15min', 
                                                        reg_method='k_fold', alpha=5, folds=5)

train_df_meem, test_df_meem = mean_encode(train_df, test_df, columns=categ_cols, target_col='dep_delayed_15min', 
                                                        reg_method='expanding_mean', alpha=5)

train_df_meem.columns = [col + 'expanding_mean' for col in categ_cols]
test_df_meem.columns = [col + 'expanding_mean' for col in categ_cols]

In [184]:
y_train = train_df['dep_delayed_15min']

X_train_full = preproc(train_df.copy())
X_test_full = preproc(test_df.copy(), test=True)

del X_train_full['dep_delayed_15min']

X_train_full = pd.concat([X_train_full, train_df_fe, train_df_mekf, train_df_meem], axis=1)
X_test_full = pd.concat([X_test_full, test_df_fe, test_df_mekf, test_df_meem], axis=1)

X_train_full.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,OriginCITY,OriginSTATE,...,freq_Dest,freq_flight,mean_dep_delayed_15min_UniqueCarrier,mean_dep_delayed_15min_Origin,mean_dep_delayed_15min_Dest,mean_dep_delayed_15min_flight,UniqueCarrierexpanding_mean,Originexpanding_mean,Destexpanding_mean,flightexpanding_mean
0,8,21,7,1174,AA,ATL,DFW,732,Atlanta,GA,...,0.04337,0.00136,0.185031,0.255794,0.153744,0.254111,0.184438,0.255592,0.153869,0.236364
1,4,20,3,948,US,PIT,MCO,834,Pittsburgh,PA,...,0.01728,0.00054,0.171243,0.168373,0.188049,0.162290,0.174815,0.125000,0.193103,0.000000
2,9,2,5,862,XE,RDU,CLE,416,Raleigh,NC,...,0.01218,0.00022,0.172890,0.191850,0.183729,0.113546,0.170219,0.176938,0.180328,0.200000
3,11,25,6,615,OO,DEN,MEM,872,Denver,CO,...,0.00629,0.00014,0.170996,0.195163,0.147418,0.122012,0.171821,0.186235,0.162791,0.333333
4,10,7,6,1108,WN,MDW,OMA,423,Chicago,IL,...,0.00311,0.00030,0.214693,0.249266,0.241468,0.412145,0.207071,0.206897,0.333333,1.000000


# CV

In [187]:
X_tr, X_valid, y_tr, y_valid = train_test_split(X_train_full, y_train, test_size=0.25, random_state=174)

In [191]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=11138)

In [192]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']
X_train = X_tr[cols_to_use]

cat_cols = ['UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74169+-0.00689


In [193]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74150+-0.00715


In [194]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'DayofMonth', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74615+-0.00699


In [196]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75272+-0.00655


In [197]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75272+-0.00655


In [198]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75308+-0.00738


In [204]:
cols_to_use = ['DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75252+-0.00679


In [205]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'dtimeLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75217+-0.00719


In [207]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75340+-0.00677


In [208]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.77118+-0.00761


In [209]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75340+-0.00677


In [206]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75383+-0.00712


In [199]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'dtimeLabel', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75230+-0.00766


In [201]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'dtimeLabel', 'hour_binsLabel', 'flight']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'flight', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.75370+-0.00730


In [202]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Dest', 'Distance_log',
              '4seasonsLabel', 'dtimeLabel', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Dest', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74437+-0.00722


In [203]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Distance_log',
              '4seasonsLabel', 'dtimeLabel', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74953+-0.00634


In [200]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Distance_log', 'flight',
              '4seasonsLabel', 'dtimeLabel', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'flight',
            'dtimeLabel', 'hour_binsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True)
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.74676+-0.00690


In [210]:
X_tr.columns

Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance', 'OriginCITY', 'OriginSTATE', 'DestCITY',
       'DestSTATE', 'flight', 'flight_city', 'flight_state', 'Holiday',
       'sin_time', 'cos_time', 'hour', 'minutes', 'morning', 'day', 'evening',
       'night', 'dtimeLabel', 'low_delay', 'other_time', 'delay_time',
       'middle_time', 'hour_bins', 'winter', 'spring', 'summer', 'autumn',
       '4seasonsLabel', 'Distance_log', 'OriginCOUNTRY', 'OriginLATITUDE',
       'OriginLONGITUDE', 'Originx_coord', 'Originy_coord', 'Originz_coord',
       'DestCOUNTRY', 'DestLATITUDE', 'DestLONGITUDE', 'Destx_coord',
       'Desty_coord', 'Destz_coord', 'x_coord', 'y_coord', 'z_coord',
       'LATITUDE', 'LONGITUDE', 'UniqueCarrierLabel', 'OriginLabel',
       'DestLabel', 'flightLabel', 'flight_cityLabel', 'flight_stateLabel',
       'hour_binsLabel', 'freq_UniqueCarrier', 'freq_Origin', 'freq_Dest',
       'freq_flight', 'mean_dep_delayed_15

In [211]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.77827+-0.00528


In [212]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.77388+-0.00506


In [214]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'OriginLabel', 'DestLabel', 'Distance_log',
              '4seasonsLabel', 'hour', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['4seasonsLabel', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'OriginLabel', 'DestLabel', 
            'dtimeLabel', 'hour_binsLabel', 'hour']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.77388+-0.00512


In [ ]:
cols_to_use = ['Dest', 'DayofMonth', 'hour', 'DepTime', 'UniqueCarrier', 'Distance', '4seasonsLabel',
               'Month', 'minutes', 'DayOfWeek', 'hour_binsLabel', 'winter', 'spring', 'summer', 'autumn']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel', 
            'summer', 'autumn', 'winter', 'spring', '4seasonsLabel']

categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')
cv_score = cross_val_score(clf, X_train, y_tr, cv=skf, scoring='roc_auc', fit_params=cat_features)
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

Check valid

In [228]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'minutes', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')
clf.fit(X_train, y_tr, cat_features=categ_feat_idx)
y_valid_pred = clf.predict_proba(X_valid[cols_to_use])[:, 1]
score = roc_auc_score(y_valid, y_valid_pred)
print(score)

0.7985530916958626


In [229]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'minutes', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 31,
         'l2_leaf_reg': 2310.1297000831582,
         'iterations': 2000,
         'depth': 6,
         'border_count': 254}

clf = CatBoostClassifier(random_state=177, task_type='GPU', **params)
clf.fit(X_train, y_tr, cat_features=categ_feat_idx)
y_valid_pred = clf.predict_proba(X_valid[cols_to_use])[:, 1]
score = roc_auc_score(y_valid, y_valid_pred)
print(score)

0:	learn: 0.6390945	total: 115ms	remaining: 3m 50s
1:	learn: 0.5948530	total: 242ms	remaining: 4m 1s
2:	learn: 0.5619304	total: 361ms	remaining: 4m
3:	learn: 0.5362781	total: 479ms	remaining: 3m 58s
4:	learn: 0.5150363	total: 586ms	remaining: 3m 53s
5:	learn: 0.4995445	total: 705ms	remaining: 3m 54s
6:	learn: 0.4865930	total: 825ms	remaining: 3m 54s
7:	learn: 0.4752243	total: 936ms	remaining: 3m 53s
8:	learn: 0.4665794	total: 1.05s	remaining: 3m 53s
9:	learn: 0.4588792	total: 1.17s	remaining: 3m 51s
10:	learn: 0.4530330	total: 1.28s	remaining: 3m 52s
11:	learn: 0.4481340	total: 1.39s	remaining: 3m 50s
12:	learn: 0.4441194	total: 1.52s	remaining: 3m 52s
13:	learn: 0.4404593	total: 1.63s	remaining: 3m 51s
14:	learn: 0.4374842	total: 1.75s	remaining: 3m 50s
15:	learn: 0.4344645	total: 1.87s	remaining: 3m 51s
16:	learn: 0.4325789	total: 1.98s	remaining: 3m 51s
17:	learn: 0.4306128	total: 2.08s	remaining: 3m 49s
18:	learn: 0.4293102	total: 2.19s	remaining: 3m 48s
19:	learn: 0.4274745	total:

159:	learn: 0.3969011	total: 18.3s	remaining: 3m 30s
160:	learn: 0.3968656	total: 18.4s	remaining: 3m 30s
161:	learn: 0.3967483	total: 18.5s	remaining: 3m 30s
162:	learn: 0.3967205	total: 18.6s	remaining: 3m 30s
163:	learn: 0.3966511	total: 18.8s	remaining: 3m 30s
164:	learn: 0.3966165	total: 18.9s	remaining: 3m 30s
165:	learn: 0.3965652	total: 19s	remaining: 3m 29s
166:	learn: 0.3965633	total: 19.1s	remaining: 3m 29s
167:	learn: 0.3965607	total: 19.2s	remaining: 3m 29s
168:	learn: 0.3964274	total: 19.3s	remaining: 3m 29s
169:	learn: 0.3964268	total: 19.4s	remaining: 3m 29s
170:	learn: 0.3963384	total: 19.5s	remaining: 3m 29s
171:	learn: 0.3963022	total: 19.7s	remaining: 3m 28s
172:	learn: 0.3962283	total: 19.8s	remaining: 3m 28s
173:	learn: 0.3961624	total: 19.9s	remaining: 3m 28s
174:	learn: 0.3960391	total: 20s	remaining: 3m 28s
175:	learn: 0.3959514	total: 20.1s	remaining: 3m 28s
176:	learn: 0.3958966	total: 20.2s	remaining: 3m 28s
177:	learn: 0.3958061	total: 20.4s	remaining: 3m 2

315:	learn: 0.3890241	total: 35.8s	remaining: 3m 10s
316:	learn: 0.3890217	total: 35.9s	remaining: 3m 10s
317:	learn: 0.3890193	total: 36s	remaining: 3m 10s
318:	learn: 0.3889823	total: 36.2s	remaining: 3m 10s
319:	learn: 0.3888976	total: 36.3s	remaining: 3m 10s
320:	learn: 0.3888276	total: 36.4s	remaining: 3m 10s
321:	learn: 0.3887780	total: 36.5s	remaining: 3m 10s
322:	learn: 0.3887636	total: 36.6s	remaining: 3m 10s
323:	learn: 0.3887180	total: 36.7s	remaining: 3m 10s
324:	learn: 0.3887176	total: 36.8s	remaining: 3m 9s
325:	learn: 0.3886765	total: 37s	remaining: 3m 9s
326:	learn: 0.3886754	total: 37.1s	remaining: 3m 9s
327:	learn: 0.3886514	total: 37.2s	remaining: 3m 9s
328:	learn: 0.3885998	total: 37.3s	remaining: 3m 9s
329:	learn: 0.3884609	total: 37.4s	remaining: 3m 9s
330:	learn: 0.3884570	total: 37.5s	remaining: 3m 9s
331:	learn: 0.3884562	total: 37.6s	remaining: 3m 8s
332:	learn: 0.3884162	total: 37.7s	remaining: 3m 8s
333:	learn: 0.3883997	total: 37.9s	remaining: 3m 8s
334:	le

474:	learn: 0.3841155	total: 53.7s	remaining: 2m 52s
475:	learn: 0.3840378	total: 53.8s	remaining: 2m 52s
476:	learn: 0.3839649	total: 53.9s	remaining: 2m 52s
477:	learn: 0.3838756	total: 54s	remaining: 2m 52s
478:	learn: 0.3838665	total: 54.1s	remaining: 2m 51s
479:	learn: 0.3838275	total: 54.3s	remaining: 2m 51s
480:	learn: 0.3838252	total: 54.4s	remaining: 2m 51s
481:	learn: 0.3837860	total: 54.5s	remaining: 2m 51s
482:	learn: 0.3837646	total: 54.6s	remaining: 2m 51s
483:	learn: 0.3837606	total: 54.7s	remaining: 2m 51s
484:	learn: 0.3837243	total: 54.9s	remaining: 2m 51s
485:	learn: 0.3836792	total: 55s	remaining: 2m 51s
486:	learn: 0.3836324	total: 55.1s	remaining: 2m 51s
487:	learn: 0.3836305	total: 55.2s	remaining: 2m 51s
488:	learn: 0.3835885	total: 55.3s	remaining: 2m 50s
489:	learn: 0.3835355	total: 55.4s	remaining: 2m 50s
490:	learn: 0.3835177	total: 55.5s	remaining: 2m 50s
491:	learn: 0.3835028	total: 55.6s	remaining: 2m 50s
492:	learn: 0.3834811	total: 55.8s	remaining: 2m 5

631:	learn: 0.3796001	total: 1m 11s	remaining: 2m 34s
632:	learn: 0.3795630	total: 1m 11s	remaining: 2m 34s
633:	learn: 0.3795523	total: 1m 11s	remaining: 2m 34s
634:	learn: 0.3795402	total: 1m 11s	remaining: 2m 34s
635:	learn: 0.3794808	total: 1m 11s	remaining: 2m 33s
636:	learn: 0.3794523	total: 1m 11s	remaining: 2m 33s
637:	learn: 0.3794258	total: 1m 12s	remaining: 2m 33s
638:	learn: 0.3793842	total: 1m 12s	remaining: 2m 33s
639:	learn: 0.3793400	total: 1m 12s	remaining: 2m 33s
640:	learn: 0.3792997	total: 1m 12s	remaining: 2m 33s
641:	learn: 0.3792971	total: 1m 12s	remaining: 2m 33s
642:	learn: 0.3792903	total: 1m 12s	remaining: 2m 33s
643:	learn: 0.3792820	total: 1m 12s	remaining: 2m 33s
644:	learn: 0.3792173	total: 1m 12s	remaining: 2m 32s
645:	learn: 0.3791784	total: 1m 12s	remaining: 2m 32s
646:	learn: 0.3791060	total: 1m 13s	remaining: 2m 32s
647:	learn: 0.3790617	total: 1m 13s	remaining: 2m 32s
648:	learn: 0.3789849	total: 1m 13s	remaining: 2m 32s
649:	learn: 0.3789717	total:

...

1241:	learn: 0.3652219	total: 2m 20s	remaining: 1m 25s
1242:	learn: 0.3651786	total: 2m 20s	remaining: 1m 25s
1243:	learn: 0.3651353	total: 2m 20s	remaining: 1m 25s
1244:	learn: 0.3651252	total: 2m 20s	remaining: 1m 25s
1245:	learn: 0.3651175	total: 2m 20s	remaining: 1m 25s
1246:	learn: 0.3651145	total: 2m 20s	remaining: 1m 24s
1247:	learn: 0.3651037	total: 2m 20s	remaining: 1m 24s
1248:	learn: 0.3650653	total: 2m 20s	remaining: 1m 24s
1249:	learn: 0.3650552	total: 2m 20s	remaining: 1m 24s
1250:	learn: 0.3649884	total: 2m 21s	remaining: 1m 24s
1251:	learn: 0.3649526	total: 2m 21s	remaining: 1m 24s
1252:	learn: 0.3649419	total: 2m 21s	remaining: 1m 24s
1253:	learn: 0.3649401	total: 2m 21s	remaining: 1m 24s
1254:	learn: 0.3649184	total: 2m 21s	remaining: 1m 24s
1255:	learn: 0.3649131	total: 2m 21s	remaining: 1m 23s
1256:	learn: 0.3649127	total: 2m 21s	remaining: 1m 23s
1257:	learn: 0.3648846	total: 2m 21s	remaining: 1m 23s
1258:	learn: 0.3648469	total: 2m 21s	remaining: 1m 23s
1259:	lear

1392:	learn: 0.3620070	total: 2m 37s	remaining: 1m 8s
1393:	learn: 0.3620062	total: 2m 37s	remaining: 1m 8s
1394:	learn: 0.3619579	total: 2m 37s	remaining: 1m 8s
1395:	learn: 0.3619562	total: 2m 37s	remaining: 1m 8s
1396:	learn: 0.3619374	total: 2m 37s	remaining: 1m 7s
1397:	learn: 0.3619314	total: 2m 37s	remaining: 1m 7s
1398:	learn: 0.3618739	total: 2m 37s	remaining: 1m 7s
1399:	learn: 0.3618235	total: 2m 37s	remaining: 1m 7s
1400:	learn: 0.3618115	total: 2m 37s	remaining: 1m 7s
1401:	learn: 0.3618102	total: 2m 38s	remaining: 1m 7s
1402:	learn: 0.3617682	total: 2m 38s	remaining: 1m 7s
1403:	learn: 0.3617505	total: 2m 38s	remaining: 1m 7s
1404:	learn: 0.3617450	total: 2m 38s	remaining: 1m 7s
1405:	learn: 0.3617231	total: 2m 38s	remaining: 1m 6s
1406:	learn: 0.3617158	total: 2m 38s	remaining: 1m 6s
1407:	learn: 0.3617122	total: 2m 38s	remaining: 1m 6s
1408:	learn: 0.3617003	total: 2m 38s	remaining: 1m 6s
1409:	learn: 0.3616615	total: 2m 38s	remaining: 1m 6s
1410:	learn: 0.3616250	total

1545:	learn: 0.3589626	total: 2m 53s	remaining: 51.1s
1546:	learn: 0.3589587	total: 2m 54s	remaining: 51s
1547:	learn: 0.3589556	total: 2m 54s	remaining: 50.8s
1548:	learn: 0.3589216	total: 2m 54s	remaining: 50.7s
1549:	learn: 0.3589118	total: 2m 54s	remaining: 50.6s
1550:	learn: 0.3589108	total: 2m 54s	remaining: 50.5s
1551:	learn: 0.3588232	total: 2m 54s	remaining: 50.4s
1552:	learn: 0.3587999	total: 2m 54s	remaining: 50.3s
1553:	learn: 0.3587964	total: 2m 54s	remaining: 50.2s
1554:	learn: 0.3587725	total: 2m 54s	remaining: 50.1s
1555:	learn: 0.3587553	total: 2m 55s	remaining: 50s
1556:	learn: 0.3587362	total: 2m 55s	remaining: 49.8s
1557:	learn: 0.3586701	total: 2m 55s	remaining: 49.7s
1558:	learn: 0.3586442	total: 2m 55s	remaining: 49.6s
1559:	learn: 0.3586217	total: 2m 55s	remaining: 49.5s
1560:	learn: 0.3585820	total: 2m 55s	remaining: 49.4s
1561:	learn: 0.3585445	total: 2m 55s	remaining: 49.3s
1562:	learn: 0.3585246	total: 2m 55s	remaining: 49.2s
1563:	learn: 0.3585165	total: 2m

1701:	learn: 0.3555726	total: 3m 11s	remaining: 33.5s
1702:	learn: 0.3555543	total: 3m 11s	remaining: 33.4s
1703:	learn: 0.3555309	total: 3m 11s	remaining: 33.3s
1704:	learn: 0.3555134	total: 3m 11s	remaining: 33.2s
1705:	learn: 0.3554879	total: 3m 12s	remaining: 33.1s
1706:	learn: 0.3554879	total: 3m 12s	remaining: 33s
1707:	learn: 0.3554660	total: 3m 12s	remaining: 32.9s
1708:	learn: 0.3554147	total: 3m 12s	remaining: 32.8s
1709:	learn: 0.3553795	total: 3m 12s	remaining: 32.6s
1710:	learn: 0.3553637	total: 3m 12s	remaining: 32.5s
1711:	learn: 0.3553631	total: 3m 12s	remaining: 32.4s
1712:	learn: 0.3553512	total: 3m 12s	remaining: 32.3s
1713:	learn: 0.3553489	total: 3m 12s	remaining: 32.2s
1714:	learn: 0.3553277	total: 3m 13s	remaining: 32.1s
1715:	learn: 0.3553258	total: 3m 13s	remaining: 32s
1716:	learn: 0.3553007	total: 3m 13s	remaining: 31.9s
1717:	learn: 0.3552904	total: 3m 13s	remaining: 31.8s
1718:	learn: 0.3552763	total: 3m 13s	remaining: 31.6s
1719:	learn: 0.3552643	total: 3m

1855:	learn: 0.3523861	total: 3m 29s	remaining: 16.2s
1856:	learn: 0.3523827	total: 3m 29s	remaining: 16.1s
1857:	learn: 0.3523504	total: 3m 29s	remaining: 16s
1858:	learn: 0.3523056	total: 3m 29s	remaining: 15.9s
1859:	learn: 0.3522735	total: 3m 29s	remaining: 15.8s
1860:	learn: 0.3522683	total: 3m 29s	remaining: 15.7s
1861:	learn: 0.3522672	total: 3m 29s	remaining: 15.5s
1862:	learn: 0.3522445	total: 3m 29s	remaining: 15.4s
1863:	learn: 0.3521973	total: 3m 30s	remaining: 15.3s
1864:	learn: 0.3521699	total: 3m 30s	remaining: 15.2s
1865:	learn: 0.3521423	total: 3m 30s	remaining: 15.1s
1866:	learn: 0.3521403	total: 3m 30s	remaining: 15s
1867:	learn: 0.3520751	total: 3m 30s	remaining: 14.9s
1868:	learn: 0.3520433	total: 3m 30s	remaining: 14.8s
1869:	learn: 0.3520023	total: 3m 30s	remaining: 14.6s
1870:	learn: 0.3519707	total: 3m 30s	remaining: 14.5s
1871:	learn: 0.3519689	total: 3m 30s	remaining: 14.4s
1872:	learn: 0.3519309	total: 3m 31s	remaining: 14.3s
1873:	learn: 0.3519211	total: 3m

In [230]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'minutes', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 49,
         'l2_leaf_reg': 259.5024211399737,
         'iterations': 3186,
         'depth': 8,
         'border_count': 254}

clf = CatBoostClassifier(random_state=177, task_type='GPU', **params)
clf.fit(X_train, y_tr, cat_features=categ_feat_idx)
y_valid_pred = clf.predict_proba(X_valid[cols_to_use])[:, 1]
score = roc_auc_score(y_valid, y_valid_pred)
print(score)

0:	learn: 0.5878049	total: 218ms	remaining: 11m 34s
1:	learn: 0.5227365	total: 440ms	remaining: 11m 41s
2:	learn: 0.4855222	total: 640ms	remaining: 11m 19s
3:	learn: 0.4637396	total: 825ms	remaining: 10m 56s
4:	learn: 0.4496878	total: 1.04s	remaining: 10m 59s
5:	learn: 0.4398768	total: 1.26s	remaining: 11m 8s
6:	learn: 0.4323550	total: 1.46s	remaining: 11m 3s
7:	learn: 0.4284638	total: 1.7s	remaining: 11m 14s
8:	learn: 0.4250646	total: 1.89s	remaining: 11m 8s
9:	learn: 0.4224835	total: 2.12s	remaining: 11m 13s
10:	learn: 0.4195822	total: 2.33s	remaining: 11m 11s
11:	learn: 0.4161237	total: 2.56s	remaining: 11m 17s
12:	learn: 0.4137251	total: 2.74s	remaining: 11m 9s
13:	learn: 0.4124173	total: 2.96s	remaining: 11m 10s
14:	learn: 0.4108587	total: 3.15s	remaining: 11m 7s
15:	learn: 0.4094243	total: 3.39s	remaining: 11m 12s
16:	learn: 0.4084426	total: 3.59s	remaining: 11m 9s
17:	learn: 0.4064951	total: 3.81s	remaining: 11m 11s
18:	learn: 0.4057792	total: 4s	remaining: 11m 6s
19:	learn: 0.4

155:	learn: 0.3619020	total: 31.8s	remaining: 10m 18s
156:	learn: 0.3616256	total: 32s	remaining: 10m 18s
157:	learn: 0.3613902	total: 32.2s	remaining: 10m 17s
158:	learn: 0.3608855	total: 32.5s	remaining: 10m 18s
159:	learn: 0.3607578	total: 32.7s	remaining: 10m 17s
160:	learn: 0.3605388	total: 32.8s	remaining: 10m 17s
161:	learn: 0.3603327	total: 33s	remaining: 10m 16s
162:	learn: 0.3600881	total: 33.3s	remaining: 10m 16s
163:	learn: 0.3598183	total: 33.4s	remaining: 10m 16s
164:	learn: 0.3596136	total: 33.6s	remaining: 10m 15s
165:	learn: 0.3595888	total: 33.8s	remaining: 10m 15s
166:	learn: 0.3590708	total: 34.1s	remaining: 10m 16s
167:	learn: 0.3587971	total: 34.3s	remaining: 10m 15s
168:	learn: 0.3584296	total: 34.5s	remaining: 10m 15s
169:	learn: 0.3580543	total: 34.7s	remaining: 10m 14s
170:	learn: 0.3577685	total: 34.9s	remaining: 10m 14s
171:	learn: 0.3574427	total: 35.1s	remaining: 10m 14s
172:	learn: 0.3571511	total: 35.3s	remaining: 10m 14s
173:	learn: 0.3568287	total: 35.

311:	learn: 0.3264779	total: 1m 3s	remaining: 9m 47s
312:	learn: 0.3262066	total: 1m 4s	remaining: 9m 47s
313:	learn: 0.3261013	total: 1m 4s	remaining: 9m 47s
314:	learn: 0.3259930	total: 1m 4s	remaining: 9m 47s
315:	learn: 0.3259844	total: 1m 4s	remaining: 9m 46s
316:	learn: 0.3257539	total: 1m 4s	remaining: 9m 46s
317:	learn: 0.3255839	total: 1m 4s	remaining: 9m 46s
318:	learn: 0.3255797	total: 1m 5s	remaining: 9m 45s
319:	learn: 0.3253035	total: 1m 5s	remaining: 9m 45s
320:	learn: 0.3250896	total: 1m 5s	remaining: 9m 45s
321:	learn: 0.3249070	total: 1m 5s	remaining: 9m 45s
322:	learn: 0.3246372	total: 1m 6s	remaining: 9m 45s
323:	learn: 0.3244466	total: 1m 6s	remaining: 9m 44s
324:	learn: 0.3240833	total: 1m 6s	remaining: 9m 44s
325:	learn: 0.3239368	total: 1m 6s	remaining: 9m 44s
326:	learn: 0.3235922	total: 1m 6s	remaining: 9m 44s
327:	learn: 0.3232894	total: 1m 7s	remaining: 9m 44s
328:	learn: 0.3230630	total: 1m 7s	remaining: 9m 44s
329:	learn: 0.3228165	total: 1m 7s	remaining: 

465:	learn: 0.2965201	total: 1m 35s	remaining: 9m 16s
466:	learn: 0.2963588	total: 1m 35s	remaining: 9m 16s
467:	learn: 0.2962741	total: 1m 35s	remaining: 9m 15s
468:	learn: 0.2959390	total: 1m 35s	remaining: 9m 15s
469:	learn: 0.2956042	total: 1m 36s	remaining: 9m 15s
470:	learn: 0.2954472	total: 1m 36s	remaining: 9m 15s
471:	learn: 0.2952758	total: 1m 36s	remaining: 9m 15s
472:	learn: 0.2951416	total: 1m 36s	remaining: 9m 14s
473:	learn: 0.2949064	total: 1m 36s	remaining: 9m 14s
474:	learn: 0.2947670	total: 1m 37s	remaining: 9m 14s
475:	learn: 0.2945372	total: 1m 37s	remaining: 9m 14s
476:	learn: 0.2943525	total: 1m 37s	remaining: 9m 14s
477:	learn: 0.2941773	total: 1m 37s	remaining: 9m 13s
478:	learn: 0.2939343	total: 1m 38s	remaining: 9m 13s
479:	learn: 0.2938154	total: 1m 38s	remaining: 9m 13s
480:	learn: 0.2936870	total: 1m 38s	remaining: 9m 13s
481:	learn: 0.2934827	total: 1m 38s	remaining: 9m 13s
482:	learn: 0.2931885	total: 1m 38s	remaining: 9m 13s
483:	learn: 0.2930191	total:

620:	learn: 0.2702850	total: 2m 7s	remaining: 8m 45s
621:	learn: 0.2698882	total: 2m 7s	remaining: 8m 44s
622:	learn: 0.2697329	total: 2m 7s	remaining: 8m 44s
623:	learn: 0.2693489	total: 2m 7s	remaining: 8m 44s
624:	learn: 0.2693189	total: 2m 7s	remaining: 8m 44s
625:	learn: 0.2691556	total: 2m 8s	remaining: 8m 44s
626:	learn: 0.2690339	total: 2m 8s	remaining: 8m 43s
627:	learn: 0.2687991	total: 2m 8s	remaining: 8m 43s
628:	learn: 0.2686942	total: 2m 8s	remaining: 8m 43s
629:	learn: 0.2686668	total: 2m 8s	remaining: 8m 43s
630:	learn: 0.2685088	total: 2m 9s	remaining: 8m 43s
631:	learn: 0.2682964	total: 2m 9s	remaining: 8m 42s
632:	learn: 0.2681601	total: 2m 9s	remaining: 8m 42s
633:	learn: 0.2680820	total: 2m 9s	remaining: 8m 42s
634:	learn: 0.2679148	total: 2m 10s	remaining: 8m 42s
635:	learn: 0.2677238	total: 2m 10s	remaining: 8m 42s
636:	learn: 0.2676123	total: 2m 10s	remaining: 8m 41s
637:	learn: 0.2674864	total: 2m 10s	remaining: 8m 41s
638:	learn: 0.2674684	total: 2m 10s	remain

...

2437:	learn: 0.0922421	total: 8m 20s	remaining: 2m 33s
2438:	learn: 0.0921528	total: 8m 21s	remaining: 2m 33s
2439:	learn: 0.0921513	total: 8m 21s	remaining: 2m 33s
2440:	learn: 0.0920984	total: 8m 21s	remaining: 2m 33s
2441:	learn: 0.0920731	total: 8m 21s	remaining: 2m 32s
2442:	learn: 0.0920057	total: 8m 21s	remaining: 2m 32s
2443:	learn: 0.0919549	total: 8m 22s	remaining: 2m 32s
2444:	learn: 0.0919154	total: 8m 22s	remaining: 2m 32s
2445:	learn: 0.0918701	total: 8m 22s	remaining: 2m 32s
2446:	learn: 0.0917966	total: 8m 22s	remaining: 2m 31s
2447:	learn: 0.0917684	total: 8m 22s	remaining: 2m 31s
2448:	learn: 0.0916550	total: 8m 23s	remaining: 2m 31s
2449:	learn: 0.0916292	total: 8m 23s	remaining: 2m 31s
2450:	learn: 0.0916040	total: 8m 23s	remaining: 2m 31s
2451:	learn: 0.0915325	total: 8m 23s	remaining: 2m 30s
2452:	learn: 0.0914997	total: 8m 23s	remaining: 2m 30s
2453:	learn: 0.0914670	total: 8m 24s	remaining: 2m 30s
2454:	learn: 0.0913916	total: 8m 24s	remaining: 2m 30s
2455:	lear

2588:	learn: 0.0857831	total: 8m 51s	remaining: 2m 2s
2589:	learn: 0.0857458	total: 8m 52s	remaining: 2m 2s
2590:	learn: 0.0856997	total: 8m 52s	remaining: 2m 2s
2591:	learn: 0.0856529	total: 8m 52s	remaining: 2m 2s
2592:	learn: 0.0856269	total: 8m 52s	remaining: 2m 1s
2593:	learn: 0.0855657	total: 8m 52s	remaining: 2m 1s
2594:	learn: 0.0855308	total: 8m 52s	remaining: 2m 1s
2595:	learn: 0.0854676	total: 8m 53s	remaining: 2m 1s
2596:	learn: 0.0854371	total: 8m 53s	remaining: 2m
2597:	learn: 0.0853669	total: 8m 53s	remaining: 2m
2598:	learn: 0.0853073	total: 8m 53s	remaining: 2m
2599:	learn: 0.0852778	total: 8m 53s	remaining: 2m
2600:	learn: 0.0852361	total: 8m 54s	remaining: 2m
2601:	learn: 0.0851977	total: 8m 54s	remaining: 1m 59s
2602:	learn: 0.0851823	total: 8m 54s	remaining: 1m 59s
2603:	learn: 0.0851582	total: 8m 54s	remaining: 1m 59s
2604:	learn: 0.0851162	total: 8m 54s	remaining: 1m 59s
2605:	learn: 0.0850778	total: 8m 55s	remaining: 1m 59s
2606:	learn: 0.0850521	total: 8m 55s	r

2739:	learn: 0.0797458	total: 9m 22s	remaining: 1m 31s
2740:	learn: 0.0797027	total: 9m 22s	remaining: 1m 31s
2741:	learn: 0.0796743	total: 9m 22s	remaining: 1m 31s
2742:	learn: 0.0796287	total: 9m 22s	remaining: 1m 30s
2743:	learn: 0.0796140	total: 9m 22s	remaining: 1m 30s
2744:	learn: 0.0795949	total: 9m 23s	remaining: 1m 30s
2745:	learn: 0.0795435	total: 9m 23s	remaining: 1m 30s
2746:	learn: 0.0795301	total: 9m 23s	remaining: 1m 30s
2747:	learn: 0.0794665	total: 9m 23s	remaining: 1m 29s
2748:	learn: 0.0794305	total: 9m 24s	remaining: 1m 29s
2749:	learn: 0.0793741	total: 9m 24s	remaining: 1m 29s
2750:	learn: 0.0793199	total: 9m 24s	remaining: 1m 29s
2751:	learn: 0.0792987	total: 9m 24s	remaining: 1m 29s
2752:	learn: 0.0792545	total: 9m 24s	remaining: 1m 28s
2753:	learn: 0.0792028	total: 9m 25s	remaining: 1m 28s
2754:	learn: 0.0791782	total: 9m 25s	remaining: 1m 28s
2755:	learn: 0.0791275	total: 9m 25s	remaining: 1m 28s
2756:	learn: 0.0790833	total: 9m 25s	remaining: 1m 28s
2757:	lear

2889:	learn: 0.0741602	total: 9m 52s	remaining: 1m
2890:	learn: 0.0741287	total: 9m 52s	remaining: 1m
2891:	learn: 0.0740918	total: 9m 53s	remaining: 1m
2892:	learn: 0.0740302	total: 9m 53s	remaining: 1m
2893:	learn: 0.0739975	total: 9m 53s	remaining: 59.9s
2894:	learn: 0.0739516	total: 9m 53s	remaining: 59.7s
2895:	learn: 0.0739369	total: 9m 53s	remaining: 59.5s
2896:	learn: 0.0739016	total: 9m 54s	remaining: 59.3s
2897:	learn: 0.0738760	total: 9m 54s	remaining: 59.1s
2898:	learn: 0.0738616	total: 9m 54s	remaining: 58.8s
2899:	learn: 0.0737977	total: 9m 54s	remaining: 58.6s
2900:	learn: 0.0737527	total: 9m 54s	remaining: 58.4s
2901:	learn: 0.0737493	total: 9m 55s	remaining: 58.2s
2902:	learn: 0.0736905	total: 9m 55s	remaining: 58s
2903:	learn: 0.0736466	total: 9m 55s	remaining: 57.8s
2904:	learn: 0.0736083	total: 9m 55s	remaining: 57.6s
2905:	learn: 0.0736070	total: 9m 55s	remaining: 57.4s
2906:	learn: 0.0735913	total: 9m 56s	remaining: 57.2s
2907:	learn: 0.0735289	total: 9m 56s	remai

3041:	learn: 0.0691836	total: 10m 23s	remaining: 29.5s
3042:	learn: 0.0691288	total: 10m 23s	remaining: 29.3s
3043:	learn: 0.0690932	total: 10m 23s	remaining: 29.1s
3044:	learn: 0.0690929	total: 10m 24s	remaining: 28.9s
3045:	learn: 0.0690510	total: 10m 24s	remaining: 28.7s
3046:	learn: 0.0690200	total: 10m 24s	remaining: 28.5s
3047:	learn: 0.0689995	total: 10m 24s	remaining: 28.3s
3048:	learn: 0.0689627	total: 10m 25s	remaining: 28.1s
3049:	learn: 0.0689195	total: 10m 25s	remaining: 27.9s
3050:	learn: 0.0689059	total: 10m 25s	remaining: 27.7s
3051:	learn: 0.0688817	total: 10m 25s	remaining: 27.5s
3052:	learn: 0.0688224	total: 10m 25s	remaining: 27.3s
3053:	learn: 0.0687678	total: 10m 26s	remaining: 27.1s
3054:	learn: 0.0687374	total: 10m 26s	remaining: 26.9s
3055:	learn: 0.0686998	total: 10m 26s	remaining: 26.6s
3056:	learn: 0.0686880	total: 10m 26s	remaining: 26.4s
3057:	learn: 0.0686807	total: 10m 26s	remaining: 26.2s
3058:	learn: 0.0686606	total: 10m 27s	remaining: 26s
3059:	learn:

In [232]:
from catboost import Pool

cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'minutes', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 49,
         'l2_leaf_reg': 259.5024211399737,
         'iterations': 3186,
         'depth': 8,
         'border_count': 254}
valid = Pool(X_valid[cols_to_use], y_valid, cat_features=categ_feat_idx)
clf = CatBoostClassifier(random_state=177, task_type='GPU', **params)
clf.fit(X_train, y_tr, cat_features=categ_feat_idx, eval_set=valid)
y_valid_pred = clf.predict_proba(X_valid[cols_to_use])[:, 1]
score = roc_auc_score(y_valid, y_valid_pred)
print(score)

0:	learn: 0.6363083	test: 0.6441355	best: 0.6441355 (0)	total: 207ms	remaining: 10m 58s
1:	learn: 0.5909856	test: 0.6021014	best: 0.6021014 (1)	total: 416ms	remaining: 11m 2s
2:	learn: 0.5571173	test: 0.5656287	best: 0.5656287 (2)	total: 622ms	remaining: 10m 59s
3:	learn: 0.5287697	test: 0.5400973	best: 0.5400973 (3)	total: 806ms	remaining: 10m 41s
4:	learn: 0.5078020	test: 0.5214923	best: 0.5214923 (4)	total: 1.01s	remaining: 10m 43s
5:	learn: 0.4910716	test: 0.5017803	best: 0.5017803 (5)	total: 1.24s	remaining: 10m 56s
6:	learn: 0.4776673	test: 0.4900486	best: 0.4900486 (6)	total: 1.44s	remaining: 10m 54s
7:	learn: 0.4678785	test: 0.4789359	best: 0.4789359 (7)	total: 1.65s	remaining: 10m 55s
8:	learn: 0.4595533	test: 0.4694228	best: 0.4694228 (8)	total: 1.85s	remaining: 10m 54s
9:	learn: 0.4523404	test: 0.4611891	best: 0.4611891 (9)	total: 2.06s	remaining: 10m 53s
10:	learn: 0.4462122	test: 0.4562942	best: 0.4562942 (10)	total: 2.26s	remaining: 10m 51s
11:	learn: 0.4410066	test: 0.44

93:	learn: 0.3875868	test: 0.4010701	best: 0.4010701 (93)	total: 19.8s	remaining: 10m 51s
94:	learn: 0.3872966	test: 0.4009818	best: 0.4009818 (94)	total: 20s	remaining: 10m 50s
95:	learn: 0.3871568	test: 0.4008533	best: 0.4008533 (95)	total: 20.2s	remaining: 10m 49s
96:	learn: 0.3868619	test: 0.4007150	best: 0.4007150 (96)	total: 20.4s	remaining: 10m 49s
97:	learn: 0.3866352	test: 0.4006663	best: 0.4006663 (97)	total: 20.6s	remaining: 10m 49s
98:	learn: 0.3864052	test: 0.4005239	best: 0.4005239 (98)	total: 20.8s	remaining: 10m 49s
99:	learn: 0.3861897	test: 0.4004913	best: 0.4004913 (99)	total: 21s	remaining: 10m 49s
100:	learn: 0.3858758	test: 0.4005618	best: 0.4004913 (99)	total: 21.2s	remaining: 10m 48s
101:	learn: 0.3857327	test: 0.4005423	best: 0.4004913 (99)	total: 21.5s	remaining: 10m 48s
102:	learn: 0.3854903	test: 0.4004715	best: 0.4004715 (102)	total: 21.7s	remaining: 10m 48s
103:	learn: 0.3854715	test: 0.4004794	best: 0.4004715 (102)	total: 21.8s	remaining: 10m 47s
104:	lea

183:	learn: 0.3730209	test: 0.3965857	best: 0.3965441 (182)	total: 38.7s	remaining: 10m 31s
184:	learn: 0.3729341	test: 0.3965392	best: 0.3965392 (184)	total: 38.9s	remaining: 10m 31s
185:	learn: 0.3729219	test: 0.3965537	best: 0.3965392 (184)	total: 39.1s	remaining: 10m 31s
186:	learn: 0.3726895	test: 0.3965212	best: 0.3965212 (186)	total: 39.4s	remaining: 10m 31s
187:	learn: 0.3725840	test: 0.3965649	best: 0.3965212 (186)	total: 39.6s	remaining: 10m 31s
188:	learn: 0.3724229	test: 0.3965216	best: 0.3965212 (186)	total: 39.8s	remaining: 10m 31s
189:	learn: 0.3722537	test: 0.3964351	best: 0.3964351 (189)	total: 40s	remaining: 10m 30s
190:	learn: 0.3721143	test: 0.3963788	best: 0.3963788 (190)	total: 40.2s	remaining: 10m 30s
191:	learn: 0.3721112	test: 0.3963807	best: 0.3963788 (190)	total: 40.4s	remaining: 10m 29s
192:	learn: 0.3719210	test: 0.3963636	best: 0.3963636 (192)	total: 40.6s	remaining: 10m 30s
193:	learn: 0.3717401	test: 0.3963925	best: 0.3963636 (192)	total: 40.8s	remaining

273:	learn: 0.3615185	test: 0.3947794	best: 0.3947794 (273)	total: 57.7s	remaining: 10m 13s
274:	learn: 0.3613758	test: 0.3947604	best: 0.3947604 (274)	total: 57.9s	remaining: 10m 13s
275:	learn: 0.3613714	test: 0.3947666	best: 0.3947604 (274)	total: 58.1s	remaining: 10m 12s
276:	learn: 0.3613410	test: 0.3947694	best: 0.3947604 (274)	total: 58.3s	remaining: 10m 12s
277:	learn: 0.3612064	test: 0.3947564	best: 0.3947564 (277)	total: 58.5s	remaining: 10m 11s
278:	learn: 0.3611161	test: 0.3946473	best: 0.3946473 (278)	total: 58.7s	remaining: 10m 11s
279:	learn: 0.3610461	test: 0.3946418	best: 0.3946418 (279)	total: 58.9s	remaining: 10m 11s
280:	learn: 0.3608637	test: 0.3946375	best: 0.3946375 (280)	total: 59.1s	remaining: 10m 11s
281:	learn: 0.3607754	test: 0.3945875	best: 0.3945875 (281)	total: 59.3s	remaining: 10m 10s
282:	learn: 0.3606723	test: 0.3945891	best: 0.3945875 (281)	total: 59.6s	remaining: 10m 10s
283:	learn: 0.3605900	test: 0.3945742	best: 0.3945742 (283)	total: 59.8s	remaini

363:	learn: 0.3512985	test: 0.3937592	best: 0.3937592 (363)	total: 1m 16s	remaining: 9m 53s
364:	learn: 0.3511638	test: 0.3936845	best: 0.3936845 (364)	total: 1m 16s	remaining: 9m 53s
365:	learn: 0.3510774	test: 0.3936581	best: 0.3936581 (365)	total: 1m 17s	remaining: 9m 53s
366:	learn: 0.3510001	test: 0.3936624	best: 0.3936581 (365)	total: 1m 17s	remaining: 9m 53s
367:	learn: 0.3507818	test: 0.3936946	best: 0.3936581 (365)	total: 1m 17s	remaining: 9m 53s
368:	learn: 0.3506546	test: 0.3936864	best: 0.3936581 (365)	total: 1m 17s	remaining: 9m 53s
369:	learn: 0.3505080	test: 0.3937043	best: 0.3936581 (365)	total: 1m 17s	remaining: 9m 53s
370:	learn: 0.3504993	test: 0.3937103	best: 0.3936581 (365)	total: 1m 18s	remaining: 9m 52s
371:	learn: 0.3504839	test: 0.3937299	best: 0.3936581 (365)	total: 1m 18s	remaining: 9m 52s
372:	learn: 0.3504434	test: 0.3936909	best: 0.3936581 (365)	total: 1m 18s	remaining: 9m 52s
373:	learn: 0.3503878	test: 0.3936075	best: 0.3936075 (373)	total: 1m 18s	remain

...

2779:	learn: 0.1627085	test: 0.3949518	best: 0.3917807 (876)	total: 9m 54s	remaining: 1m 26s
2780:	learn: 0.1626432	test: 0.3949473	best: 0.3917807 (876)	total: 9m 54s	remaining: 1m 26s
2781:	learn: 0.1625904	test: 0.3949634	best: 0.3917807 (876)	total: 9m 54s	remaining: 1m 26s
2782:	learn: 0.1625054	test: 0.3949629	best: 0.3917807 (876)	total: 9m 54s	remaining: 1m 26s
2783:	learn: 0.1624440	test: 0.3949905	best: 0.3917807 (876)	total: 9m 55s	remaining: 1m 25s
2784:	learn: 0.1624179	test: 0.3949730	best: 0.3917807 (876)	total: 9m 55s	remaining: 1m 25s
2785:	learn: 0.1623809	test: 0.3949719	best: 0.3917807 (876)	total: 9m 55s	remaining: 1m 25s
2786:	learn: 0.1623206	test: 0.3949930	best: 0.3917807 (876)	total: 9m 55s	remaining: 1m 25s
2787:	learn: 0.1622716	test: 0.3950096	best: 0.3917807 (876)	total: 9m 55s	remaining: 1m 25s
2788:	learn: 0.1621923	test: 0.3950284	best: 0.3917807 (876)	total: 9m 56s	remaining: 1m 24s
2789:	learn: 0.1621628	test: 0.3950248	best: 0.3917807 (876)	total: 9m

2868:	learn: 0.1584485	test: 0.3954654	best: 0.3917807 (876)	total: 10m 13s	remaining: 1m 7s
2869:	learn: 0.1583786	test: 0.3954498	best: 0.3917807 (876)	total: 10m 13s	remaining: 1m 7s
2870:	learn: 0.1583155	test: 0.3954395	best: 0.3917807 (876)	total: 10m 13s	remaining: 1m 7s
2871:	learn: 0.1582565	test: 0.3954130	best: 0.3917807 (876)	total: 10m 13s	remaining: 1m 7s
2872:	learn: 0.1581992	test: 0.3954085	best: 0.3917807 (876)	total: 10m 14s	remaining: 1m 6s
2873:	learn: 0.1581665	test: 0.3954353	best: 0.3917807 (876)	total: 10m 14s	remaining: 1m 6s
2874:	learn: 0.1581480	test: 0.3954431	best: 0.3917807 (876)	total: 10m 14s	remaining: 1m 6s
2875:	learn: 0.1580923	test: 0.3954567	best: 0.3917807 (876)	total: 10m 14s	remaining: 1m 6s
2876:	learn: 0.1580264	test: 0.3954557	best: 0.3917807 (876)	total: 10m 15s	remaining: 1m 6s
2877:	learn: 0.1579745	test: 0.3954732	best: 0.3917807 (876)	total: 10m 15s	remaining: 1m 5s
2878:	learn: 0.1579325	test: 0.3954817	best: 0.3917807 (876)	total: 10

2957:	learn: 0.1541247	test: 0.3955796	best: 0.3917807 (876)	total: 10m 32s	remaining: 48.8s
2958:	learn: 0.1540799	test: 0.3956027	best: 0.3917807 (876)	total: 10m 32s	remaining: 48.5s
2959:	learn: 0.1540167	test: 0.3956437	best: 0.3917807 (876)	total: 10m 32s	remaining: 48.3s
2960:	learn: 0.1539520	test: 0.3956668	best: 0.3917807 (876)	total: 10m 33s	remaining: 48.1s
2961:	learn: 0.1538890	test: 0.3956351	best: 0.3917807 (876)	total: 10m 33s	remaining: 47.9s
2962:	learn: 0.1538403	test: 0.3956409	best: 0.3917807 (876)	total: 10m 33s	remaining: 47.7s
2963:	learn: 0.1538135	test: 0.3956388	best: 0.3917807 (876)	total: 10m 33s	remaining: 47.5s
2964:	learn: 0.1537692	test: 0.3956423	best: 0.3917807 (876)	total: 10m 34s	remaining: 47.3s
2965:	learn: 0.1537287	test: 0.3956356	best: 0.3917807 (876)	total: 10m 34s	remaining: 47s
2966:	learn: 0.1537169	test: 0.3956311	best: 0.3917807 (876)	total: 10m 34s	remaining: 46.8s
2967:	learn: 0.1536731	test: 0.3956194	best: 0.3917807 (876)	total: 10m 

3046:	learn: 0.1501443	test: 0.3961425	best: 0.3917807 (876)	total: 10m 51s	remaining: 29.7s
3047:	learn: 0.1501038	test: 0.3961291	best: 0.3917807 (876)	total: 10m 51s	remaining: 29.5s
3048:	learn: 0.1500353	test: 0.3961360	best: 0.3917807 (876)	total: 10m 52s	remaining: 29.3s
3049:	learn: 0.1499995	test: 0.3961168	best: 0.3917807 (876)	total: 10m 52s	remaining: 29.1s
3050:	learn: 0.1499469	test: 0.3961436	best: 0.3917807 (876)	total: 10m 52s	remaining: 28.9s
3051:	learn: 0.1498899	test: 0.3961502	best: 0.3917807 (876)	total: 10m 52s	remaining: 28.7s
3052:	learn: 0.1498289	test: 0.3961461	best: 0.3917807 (876)	total: 10m 52s	remaining: 28.4s
3053:	learn: 0.1497499	test: 0.3961478	best: 0.3917807 (876)	total: 10m 53s	remaining: 28.2s
3054:	learn: 0.1497226	test: 0.3961387	best: 0.3917807 (876)	total: 10m 53s	remaining: 28s
3055:	learn: 0.1496894	test: 0.3961621	best: 0.3917807 (876)	total: 10m 53s	remaining: 27.8s
3056:	learn: 0.1496576	test: 0.3961276	best: 0.3917807 (876)	total: 10m 

3135:	learn: 0.1458668	test: 0.3961877	best: 0.3917807 (876)	total: 11m 10s	remaining: 10.7s
3136:	learn: 0.1458317	test: 0.3961976	best: 0.3917807 (876)	total: 11m 11s	remaining: 10.5s
3137:	learn: 0.1458092	test: 0.3962105	best: 0.3917807 (876)	total: 11m 11s	remaining: 10.3s
3138:	learn: 0.1457671	test: 0.3962316	best: 0.3917807 (876)	total: 11m 11s	remaining: 10.1s
3139:	learn: 0.1457265	test: 0.3962197	best: 0.3917807 (876)	total: 11m 11s	remaining: 9.84s
3140:	learn: 0.1456872	test: 0.3962282	best: 0.3917807 (876)	total: 11m 11s	remaining: 9.63s
3141:	learn: 0.1456607	test: 0.3962326	best: 0.3917807 (876)	total: 11m 12s	remaining: 9.41s
3142:	learn: 0.1456593	test: 0.3962303	best: 0.3917807 (876)	total: 11m 12s	remaining: 9.2s
3143:	learn: 0.1456310	test: 0.3962039	best: 0.3917807 (876)	total: 11m 12s	remaining: 8.98s
3144:	learn: 0.1456026	test: 0.3962176	best: 0.3917807 (876)	total: 11m 12s	remaining: 8.77s
3145:	learn: 0.1455396	test: 0.3962277	best: 0.3917807 (876)	total: 11m

In [233]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance_log',
              '4seasonsLabel', 'hour', 'minutes', 'hour_binsLabel']
X_train = X_tr[cols_to_use]

cat_cols = ['DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel']
categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 31,
         'l2_leaf_reg': 2310.1297000831582,
         'iterations': 2000,
         'depth': 6,
         'border_count': 254}
valid = Pool(X_valid[cols_to_use], y_valid, cat_features=categ_feat_idx)
clf = CatBoostClassifier(random_state=177, task_type='GPU', **params)
clf.fit(X_train, y_tr, cat_features=categ_feat_idx, eval_set=valid, early_stopping_rounds=20)
y_valid_pred = clf.predict_proba(X_valid[cols_to_use])[:, 1]
score = roc_auc_score(y_valid, y_valid_pred)
print(score)

0:	learn: 0.6390945	test: 0.6397233	best: 0.6397233 (0)	total: 106ms	remaining: 3m 31s
1:	learn: 0.5948529	test: 0.5979844	best: 0.5979844 (1)	total: 229ms	remaining: 3m 49s
2:	learn: 0.5619305	test: 0.5669617	best: 0.5669617 (2)	total: 354ms	remaining: 3m 55s
3:	learn: 0.5362781	test: 0.5400208	best: 0.5400208 (3)	total: 471ms	remaining: 3m 54s
4:	learn: 0.5151397	test: 0.5204877	best: 0.5204877 (4)	total: 585ms	remaining: 3m 53s
5:	learn: 0.4993312	test: 0.5040089	best: 0.5040089 (5)	total: 708ms	remaining: 3m 55s
6:	learn: 0.4863983	test: 0.4916227	best: 0.4916227 (6)	total: 845ms	remaining: 4m
7:	learn: 0.4750556	test: 0.4814498	best: 0.4814498 (7)	total: 961ms	remaining: 3m 59s
8:	learn: 0.4664181	test: 0.4713588	best: 0.4713588 (8)	total: 1.09s	remaining: 4m 1s
9:	learn: 0.4587466	test: 0.4649965	best: 0.4649965 (9)	total: 1.21s	remaining: 4m
10:	learn: 0.4532805	test: 0.4584693	best: 0.4584693 (10)	total: 1.34s	remaining: 4m 2s
11:	learn: 0.4484996	test: 0.4526972	best: 0.452697

94:	learn: 0.4019684	test: 0.4048509	best: 0.4048509 (94)	total: 11.5s	remaining: 3m 50s
95:	learn: 0.4018003	test: 0.4045441	best: 0.4045441 (95)	total: 11.6s	remaining: 3m 50s
96:	learn: 0.4016223	test: 0.4044572	best: 0.4044572 (96)	total: 11.7s	remaining: 3m 50s
97:	learn: 0.4015234	test: 0.4043079	best: 0.4043079 (97)	total: 11.8s	remaining: 3m 49s
98:	learn: 0.4014626	test: 0.4042901	best: 0.4042901 (98)	total: 12s	remaining: 3m 49s
99:	learn: 0.4014595	test: 0.4042835	best: 0.4042835 (99)	total: 12.1s	remaining: 3m 49s
100:	learn: 0.4013271	test: 0.4042852	best: 0.4042835 (99)	total: 12.2s	remaining: 3m 49s
101:	learn: 0.4012289	test: 0.4042296	best: 0.4042296 (101)	total: 12.3s	remaining: 3m 49s
102:	learn: 0.4011202	test: 0.4041092	best: 0.4041092 (102)	total: 12.5s	remaining: 3m 49s
103:	learn: 0.4010791	test: 0.4040689	best: 0.4040689 (103)	total: 12.6s	remaining: 3m 49s
104:	learn: 0.4009448	test: 0.4039624	best: 0.4039624 (104)	total: 12.7s	remaining: 3m 48s
105:	learn: 0.

186:	learn: 0.3952886	test: 0.4001739	best: 0.4001739 (186)	total: 22.5s	remaining: 3m 38s
187:	learn: 0.3951515	test: 0.3998939	best: 0.3998939 (187)	total: 22.6s	remaining: 3m 38s
188:	learn: 0.3950503	test: 0.3997039	best: 0.3997039 (188)	total: 22.8s	remaining: 3m 38s
189:	learn: 0.3949435	test: 0.3997167	best: 0.3997039 (188)	total: 22.9s	remaining: 3m 38s
190:	learn: 0.3948770	test: 0.3996972	best: 0.3996972 (190)	total: 23s	remaining: 3m 38s
191:	learn: 0.3947619	test: 0.3997180	best: 0.3996972 (190)	total: 23.1s	remaining: 3m 37s
192:	learn: 0.3946864	test: 0.3996107	best: 0.3996107 (192)	total: 23.3s	remaining: 3m 37s
193:	learn: 0.3945968	test: 0.3996151	best: 0.3996107 (192)	total: 23.4s	remaining: 3m 37s
194:	learn: 0.3944959	test: 0.3995064	best: 0.3995064 (194)	total: 23.5s	remaining: 3m 37s
195:	learn: 0.3944417	test: 0.3995288	best: 0.3995064 (194)	total: 23.6s	remaining: 3m 37s
196:	learn: 0.3944412	test: 0.3995319	best: 0.3995064 (194)	total: 23.7s	remaining: 3m 37s
1

278:	learn: 0.3900840	test: 0.3972617	best: 0.3972599 (277)	total: 33.7s	remaining: 3m 27s
279:	learn: 0.3900766	test: 0.3972538	best: 0.3972538 (279)	total: 33.8s	remaining: 3m 27s
280:	learn: 0.3900760	test: 0.3972571	best: 0.3972538 (279)	total: 33.9s	remaining: 3m 27s
281:	learn: 0.3900194	test: 0.3972560	best: 0.3972538 (279)	total: 34s	remaining: 3m 27s
282:	learn: 0.3900063	test: 0.3972416	best: 0.3972416 (282)	total: 34.1s	remaining: 3m 26s
283:	learn: 0.3899418	test: 0.3972355	best: 0.3972355 (283)	total: 34.2s	remaining: 3m 26s
284:	learn: 0.3899006	test: 0.3972432	best: 0.3972355 (283)	total: 34.3s	remaining: 3m 26s
285:	learn: 0.3898480	test: 0.3970908	best: 0.3970908 (285)	total: 34.4s	remaining: 3m 26s
286:	learn: 0.3898446	test: 0.3970894	best: 0.3970894 (286)	total: 34.6s	remaining: 3m 26s
287:	learn: 0.3898419	test: 0.3970886	best: 0.3970886 (287)	total: 34.7s	remaining: 3m 26s
288:	learn: 0.3898114	test: 0.3970588	best: 0.3970588 (288)	total: 34.8s	remaining: 3m 25s
2

370:	learn: 0.3866694	test: 0.3958238	best: 0.3958238 (370)	total: 44.4s	remaining: 3m 14s
371:	learn: 0.3866689	test: 0.3958269	best: 0.3958238 (370)	total: 44.5s	remaining: 3m 14s
372:	learn: 0.3866017	test: 0.3957329	best: 0.3957329 (372)	total: 44.6s	remaining: 3m 14s
373:	learn: 0.3865288	test: 0.3957192	best: 0.3957192 (373)	total: 44.7s	remaining: 3m 14s
374:	learn: 0.3864843	test: 0.3956913	best: 0.3956913 (374)	total: 44.8s	remaining: 3m 14s
375:	learn: 0.3864819	test: 0.3956863	best: 0.3956863 (375)	total: 44.9s	remaining: 3m 14s
376:	learn: 0.3864720	test: 0.3956859	best: 0.3956859 (376)	total: 45.1s	remaining: 3m 14s
377:	learn: 0.3864029	test: 0.3956472	best: 0.3956472 (377)	total: 45.2s	remaining: 3m 13s
378:	learn: 0.3864022	test: 0.3956500	best: 0.3956472 (377)	total: 45.3s	remaining: 3m 13s
379:	learn: 0.3863547	test: 0.3956201	best: 0.3956201 (379)	total: 45.4s	remaining: 3m 13s
380:	learn: 0.3863496	test: 0.3956187	best: 0.3956187 (380)	total: 45.5s	remaining: 3m 13s

462:	learn: 0.3837661	test: 0.3948136	best: 0.3948136 (462)	total: 55.3s	remaining: 3m 3s
463:	learn: 0.3837639	test: 0.3948084	best: 0.3948084 (463)	total: 55.4s	remaining: 3m 3s
464:	learn: 0.3837189	test: 0.3948075	best: 0.3948075 (464)	total: 55.5s	remaining: 3m 3s
465:	learn: 0.3837163	test: 0.3948106	best: 0.3948075 (464)	total: 55.6s	remaining: 3m 3s
466:	learn: 0.3836638	test: 0.3948319	best: 0.3948075 (464)	total: 55.8s	remaining: 3m 3s
467:	learn: 0.3836227	test: 0.3948039	best: 0.3948039 (467)	total: 55.9s	remaining: 3m 2s
468:	learn: 0.3835416	test: 0.3947606	best: 0.3947606 (468)	total: 56s	remaining: 3m 2s
469:	learn: 0.3835252	test: 0.3947301	best: 0.3947301 (469)	total: 56.1s	remaining: 3m 2s
470:	learn: 0.3835224	test: 0.3947299	best: 0.3947299 (470)	total: 56.2s	remaining: 3m 2s
471:	learn: 0.3835179	test: 0.3947396	best: 0.3947299 (470)	total: 56.4s	remaining: 3m 2s
472:	learn: 0.3834618	test: 0.3947326	best: 0.3947299 (470)	total: 56.5s	remaining: 3m 2s
473:	learn: 

554:	learn: 0.3810630	test: 0.3939480	best: 0.3939480 (554)	total: 1m 6s	remaining: 2m 52s
555:	learn: 0.3810246	test: 0.3939134	best: 0.3939134 (555)	total: 1m 6s	remaining: 2m 52s
556:	learn: 0.3809800	test: 0.3938724	best: 0.3938724 (556)	total: 1m 6s	remaining: 2m 52s
557:	learn: 0.3809686	test: 0.3938824	best: 0.3938724 (556)	total: 1m 6s	remaining: 2m 52s
558:	learn: 0.3809239	test: 0.3938664	best: 0.3938664 (558)	total: 1m 6s	remaining: 2m 52s
559:	learn: 0.3809233	test: 0.3938671	best: 0.3938664 (558)	total: 1m 6s	remaining: 2m 52s
560:	learn: 0.3808938	test: 0.3938362	best: 0.3938362 (560)	total: 1m 7s	remaining: 2m 51s
561:	learn: 0.3808165	test: 0.3938165	best: 0.3938165 (561)	total: 1m 7s	remaining: 2m 51s
562:	learn: 0.3807818	test: 0.3938042	best: 0.3938042 (562)	total: 1m 7s	remaining: 2m 51s
563:	learn: 0.3807813	test: 0.3938043	best: 0.3938042 (562)	total: 1m 7s	remaining: 2m 51s
564:	learn: 0.3807785	test: 0.3938067	best: 0.3938042 (562)	total: 1m 7s	remaining: 2m 51s

644:	learn: 0.3787203	test: 0.3931568	best: 0.3931397 (636)	total: 1m 17s	remaining: 2m 41s
645:	learn: 0.3786834	test: 0.3931540	best: 0.3931397 (636)	total: 1m 17s	remaining: 2m 41s
646:	learn: 0.3786514	test: 0.3931363	best: 0.3931363 (646)	total: 1m 17s	remaining: 2m 41s
647:	learn: 0.3786187	test: 0.3931341	best: 0.3931341 (647)	total: 1m 17s	remaining: 2m 41s
648:	learn: 0.3785646	test: 0.3931614	best: 0.3931341 (647)	total: 1m 17s	remaining: 2m 41s
649:	learn: 0.3785460	test: 0.3931609	best: 0.3931341 (647)	total: 1m 17s	remaining: 2m 41s
650:	learn: 0.3784828	test: 0.3931414	best: 0.3931341 (647)	total: 1m 17s	remaining: 2m 41s
651:	learn: 0.3784673	test: 0.3931445	best: 0.3931341 (647)	total: 1m 17s	remaining: 2m 41s
652:	learn: 0.3784182	test: 0.3931414	best: 0.3931341 (647)	total: 1m 18s	remaining: 2m 40s
653:	learn: 0.3784173	test: 0.3931405	best: 0.3931341 (647)	total: 1m 18s	remaining: 2m 40s
654:	learn: 0.3783668	test: 0.3931263	best: 0.3931263 (654)	total: 1m 18s	remain

In [234]:
clf.get_best_iteration()

673

GSCV

In [ ]:
clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU')

# Train

In [235]:
cols_to_use = ['Dest', 'DayofMonth', 'hour', 'DepTime', 'UniqueCarrier', 'Distance', '4seasonsLabel',
               'Month', 'minutes', 'DayOfWeek', 'hour_binsLabel', 'winter', 'spring', 'summer', 'autumn']
X_train = X_tr[cols_to_use]

cat_cols = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 
            'dtimeLabel', 'hour_binsLabel', 'hour', 'minutes', '4seasonsLabel', 
            'summer', 'autumn', 'winter', 'spring', '4seasonsLabel']

categ_feat_idx = [i for i, col in enumerate(cols_to_use) if col in cat_cols]
cat_features = {'cat_features': categ_feat_idx}

In [236]:
params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 49,
         'l2_leaf_reg': 259.5024211399737,
         'iterations': 850,
         'depth': 8,
         'border_count': 254}

clf = CatBoostClassifier(random_state=177, silent=True, task_type='GPU', **params)
clf.fit(X_train_full[cols_to_use], y_train, cat_features=categ_feat_idx)

In [237]:
ctb_test_pred = clf.predict_proba(X_test_full[cols_to_use])[:, 1]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv(PATH_TO_DATA / 'catboost_more_cats_opt.csv')